# IANNwTF HW 4
## Group 10

The following contains our solution to the exercises in IANNwTF HW 04. A Jupyter notebook versus a module format was chosen this time for purposes of organization.

### Assigment 1: Reviews
We review the homeworks for Groups 15 and 32.

### Assignment 2: MNIST Math

### 2.1 Preparing the MNIST Math Dataset

In [614]:
# Needed Imports
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense
import tqdm
import datetime
import matplotlib.pyplot as plt
import numpy as np


In [615]:
# 2.1 Load Dataset
(train_ds, test_ds), ds_info = tfds.load ('mnist', split =['train', 'test'], as_supervised = True, with_info = True)

# Info on the dataset (refresher)
# print("ds_info: \n", ds_info)
# tfds.show_examples(train_ds, ds_info)

In [616]:
# 2.2 Data Pipeline
def prepare_data(dataset, batchsize):

    '''
    :param dataset: the dataset to be prepared for input into the network
    :return: 2 datasets, one each for each of the math problems defined (see below), created after the original database was preprocessed with the
    steps below
    '''

    # Step One - General Preprocessing

    # convert data from uint8 to float32
    dataset = dataset.map(lambda img, target: (tf.cast(img, tf.float32), target))

    # flatten the images into vectors
    dataset = dataset.map(lambda img, target: (tf.reshape(img, (-1,)), target))

    # input normalization, just bringing image values from range [0, 255] to [-1, 1]
    dataset = dataset.map(lambda img, target: ((img / 128.) - 1., target))

    # Step 2 - Pairing Data Tuples & Respective Parameterized Targets

    # create a dataset that contains 2000 samples from the overall dataset paired with 2000 other samples
    data = tf.data.Dataset.zip((dataset.shuffle(2000), dataset.shuffle(2000)))

    # create the dataset for the first math problem (a + b >= 5) - remembering to cast to int versus boolean!
    greateqfive = data.map(lambda x1, x2: (x1[0], x2[0], x1[1]+x2[1]>=5))
    greateqfive = greateqfive.map(lambda x1, x2, t: (x1, x2, tf.cast(t, tf.int32)))

    # create the dataset for the second math problem (a - b = y)
    subtr = data.map(lambda x1, x2: (x1[0], x2[0], x1[1]-x2[1]))

    # Step 3 - Batching & Prefetching

    # run batching and prefetching for both datasets
    greateqfive = greateqfive.batch(batchsize)
    greateqfive = greateqfive.prefetch(tf.data.AUTOTUNE)
    subtr = subtr.batch(batchsize)
    subtr = subtr.prefetch(tf.data.AUTOTUNE)

    # return BOTH datasets
    return greateqfive, subtr

In [617]:
# Check data pipeline by examining one example from each of the four created datasets (one for each math problem for train and test)

train_ds_gef, train_ds_subtr = prepare_data(train_ds, batchsize = 32)
test_ds_gef, test_ds_subtr = prepare_data(test_ds, batchsize = 32)

for img1, img2, label in train_ds_gef.take(1):
    print(img1.shape, img2.shape, label.shape)

for img1, img2, label in train_ds_subtr.take(1):
    print(img1.shape, img2.shape, label.shape)

for img1, img2, label in test_ds_gef.take(1):
    print(img1.shape, img2.shape, label.shape)

for img1, img2, label in test_ds_subtr.take(1):
    print(img1.shape, img2.shape, label.shape)


(32, 784) (32, 784) (32,)
(32, 784) (32, 784) (32,)
(32, 784) (32, 784) (32,)
(32, 784) (32, 784) (32,)


### Assignment 3: Building Shared Weight Models

In [618]:
class MyModel(tf.keras.Model):
    def __init__(self, subtask, optimizer): # numlayers, subtask):

        '''
        param: numlayers - the desired number of hidden layers
        param: subtask - the subtask the network is being asked to solve (relevant for output layer)
        '''
        super(MyModel, self).__init__()

        self.optimizer = optimizer
        self.subtask = subtask
        # self.numlayers = numlayers
        # self.layer_list = []

        # self.flatten = tf.keras.layers.Flatten()
        
        # create 2 hidden layers with 256 units and ReLU as the activation function
        self.hidden_layer_1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.hidden_layer_2 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)

        # add desired number of hidden layers
        # for i in range(numlayers):
        #     self.layer_list.append(tf.keras.layers.Dense(units=256, activation=tf.nn.relu))
        
        if subtask == 0:
            self.output_layer = tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid)
            self.loss_function = tf.keras.losses.BinaryCrossentropy()
        elif subtask == 1:
            self.output_layer = tf.keras.layers.Dense(units=1, activation=tf.nn.softmax) # not 10 units, since the label.shape is (32,) not (32,10)
            self.loss_function = tf.keras.losses.MeanSquaredError()
        
        self.metrics_list = [
                    tf.keras.metrics.Mean(name="loss"),
                    tf.keras.metrics.BinaryAccuracy(name="acc"), # only for subtask 0, not for subtask 1
                    # tf.keras.metrics.TopKCategoricalAccuracy(3,name="top-3-acc") 
                    ]

    @tf.function
    def __call__(self, input: tuple, training = False):
        
        # # feed both inputs seperatedly into a layer, then concatenate the results before passing activity to the next layer
        # i1 = self.flatten(input[0])
        i1 = self.hidden_layer_1(input[0])
        i1 = self.hidden_layer_2(i1)

        # i2 = self.flatten(input[1])
        i2 = self.hidden_layer_1(input[1])
        i2 = self.hidden_layer_2(i2)

        i = tf.concat([i1, i2], axis=1)    

        # i1 = self.flatten(input[0])
        # i2 = self.flatten(input[1])
        
        # feed the activity through the network UP TO the output layer
        # for numlayers in self.layer_list:
        #     if numlayers == 0:
        #         i1 = self.hidden_layer_1(i1)
        #         i2 = self.hidden_layer_1(i2)
        #         i = tf.concat([i1, i2], axis=1)   # e.g. axis=1: (32,784) + (32,784) -> (32, 1568)
        #     else:
        #         i = self.layer_list[numlayers](i)

        out = self.output_layer(i)

        return out

    @property
    def metrics(self):
        return self.metrics_list

    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()

    @tf.function
    def train_step(self, input):
        img1, img2, label = input

        with tf.GradientTape() as tape:
            prediction = self((img1, img2), training=True)
            loss = self.loss_function(label, prediction)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # update loss metric
        self.metrics[0].update_state(loss)
        
        # for all metrics except loss, update states (accuracy etc.)
        for metric in self.metrics[1:]:
            metric.update_state(label, prediction) # + tf.reduce_sum(self.losses)

        # Return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    @tf.function
    def test_step(self, input):

        img1, img2, label = input

        prediction = self((img1, img2), training=False)
        loss = self.loss_function(label, prediction) # + tf.reduce_sum(self.losses)

        # update loss metric
        self.metrics[0].update_state(loss)

        # for accuracy metrics:
        for metric in self.metrics[1:]:
            metric.update_state(label, prediction)

        # Return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}


### Assignment 4: Training the Networks

In [619]:
# Initiate the logs and metrics
config_name= "config_name"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

train_log_path = f"logs/{config_name}/{current_time}/train"
val_log_path = f"logs/{config_name}/{current_time}/val"

# log writer for training metrics
train_summary_writer = tf.summary.create_file_writer(train_log_path)

# log writer for validation metrics
val_summary_writer = tf.summary.create_file_writer(val_log_path)

# Initiate epochs and learning rate as global variables
epochs = 2 # 10
learning_rate = 0.1 # 0.01

# Define arrays for saving values for later visualization
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

In [624]:
def training(subtask: int, optimizer): # numlayers: int, ):#, optimizer):
    '''
    :param subtask: defines the subtask to be solved, 0 is a + b >= 5, 1 is a - b = y
    :param optimizer: the optimizer function to use
    :return:
    '''

    # Initiate a model with the requested parameters
    network = MyModel(subtask, optimizer) # numlayers, subtask)

    # Initialize the train and test datasets, and the loss function, based on the subtask
    if subtask == 0:
        train = train_ds_gef
        test = test_ds_gef

    else:
        train = train_ds_subtr
        test = test_ds_subtr

    # Train the model
    for epoch in range(epochs):
        print(f"Starting Epoch {epoch}")

        for data in tqdm.tqdm(train, position=0, leave=True):
            metrics = network.train_step(data)

            with train_summary_writer.as_default():
                for metric in network.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

        # print the end acc and loss
        print([f"train_{key}: {value.numpy()}" for (key, value) in metrics.items()])
        

        # wooki's struggle to make a list for the visualization (pls don't delete)
        print(metrics)
        train_losses.append(metrics.values.numpy()[1])
        train_accuracies.append(metrics.values.numpy()[1])
        # for (key, value) in metrics:

            # if key == "loss":
            #     train_losses.append(value.numpy())
            # elif key == "acc":
            #     train_accuracies.append(value.numpy())
            
        # pls don't delete until here

        # reset metrics for next round
        network.reset_metrics()

        # Testing
        for data in test:
            metrics = network.test_step(data)

            # log the accs and losses
            with val_summary_writer.as_default():
                for metric in network.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step = epoch)

        # print the end acc and loss
        print([f"val_{key}: {value.numpy()}" for (key, value) in metrics.items()])

        # reset all metrics
        network.reset_metrics()


In [625]:
# Train a model to solve the first math problem
training(0, tf.keras.optimizers.Adam())
print(train_losses)
# training(0, tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.0))
# training(0, tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.5))
# training(0, tf.keras.optimizers.RMSprop())
# training(0, tf.keras.optimizers.Adagrad())

Starting Epoch 0


100%|██████████| 1875/1875 [00:12<00:00, 152.26it/s]

['train_loss: 0.18405906856060028', 'train_acc: 0.9276333451271057']
{'loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.18405907>, 'acc': <tf.Tensor: shape=(), dtype=float32, numpy=0.92763335>}


AttributeError: 'builtin_function_or_method' object has no attribute 'numpy'

In [ ]:
# Train a model to solve the second math problem
training(1, tf.keras.optimizers.Adam())
training(1, tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.0))
training(1, tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.5))
training(1, tf.keras.optimizers.RMSprop())
training(1, tf.keras.optimizers.Adagrad())

Starting Epoch 0


100%|██████████| 1875/1875 [00:11<00:00, 161.80it/s]


['train_loss: 17.700599670410156', 'train_acc: 0.08908333629369736']
{'loss': <tf.Tensor: shape=(), dtype=float32, numpy=17.7006>, 'acc': <tf.Tensor: shape=(), dtype=float32, numpy=0.08908334>}
['val_loss: 17.71076202392578', 'val_acc: 0.09120000153779984']
Starting Epoch 1


100%|██████████| 1875/1875 [00:11<00:00, 169.28it/s]


['train_loss: 17.576066970825195', 'train_acc: 0.08963333070278168']
{'loss': <tf.Tensor: shape=(), dtype=float32, numpy=17.576067>, 'acc': <tf.Tensor: shape=(), dtype=float32, numpy=0.08963333>}
['val_loss: 17.827476501464844', 'val_acc: 0.09200000017881393']
Starting Epoch 0


100%|██████████| 1875/1875 [00:11<00:00, 168.82it/s]


['train_loss: 17.7457332611084', 'train_acc: 0.08971666544675827']
{'loss': <tf.Tensor: shape=(), dtype=float32, numpy=17.745733>, 'acc': <tf.Tensor: shape=(), dtype=float32, numpy=0.089716665>}
['val_loss: 17.786941528320312', 'val_acc: 0.09319999814033508']
Starting Epoch 1


 50%|█████     | 944/1875 [00:05<00:05, 163.16it/s]


KeyboardInterrupt: 

### the loss is too high and the acc is too low for the 2nd math prob, need to figure out why & visualization is not done yet

### Assignment 5 - Experiments

Run training w/ classic SGD (no momentum)

Run training w/ Adam

Run training w/ SGD + Momentum

Run training w/ RMSrop

Run training w/ AdaGrad

In [ ]:
# Visualize the results of the above training runs

# NEED TO BE WORKED ON
fig, axs = plt.subplots(5) 
fig.suptitle('Vertically stacked subplots')
x = np.linspace(0, epochs)
y = np.sin(x ** 2)
ax1.plot(x, y)
ax2.plot(x, -y)

line1, = plt.plot(train_losses)
line2, = plt.plot(test_losses)
line3, = plt.plot(train_accuracies)
line4, = plt.plot(test_accuracies)
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend((line1, line2, line3, line4),("Training Loss", "Test Loss", "Training Accuracy", "Test Accuracy"))
fig.savefig("Title-Of-The-Figure")
plt.show()


NameError: name 'plt' is not defined